In [2]:
import pandas as pd 
df_user = pd.read_csv('./Data/df_user.csv')
df_ai_news_id = pd.read_csv('./Data/ai_news_id_title.csv') # to add ai_news_id
df_news = pd.read_csv('./Data/df_news.csv')

In [3]:
ai_news_id = df_ai_news_id['News ID'].tolist()
df_news['is_ai_news'] = df_news['News ID'].apply(lambda x: 1 if x in ai_news_id else 0)


In [4]:
df_news['is_ai_news'].value_counts()

is_ai_news
0    51261
1       21
Name: count, dtype: int64

In [5]:
df_user['Time'] = pd.to_datetime(df_user['Time'], errors='coerce')
df_user.head()

,Impression ID,User ID,Time,History,Impressions
0,1,U13740,2019-11-11 09:05:58,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,2019-11-12 18:11:30,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,2019-11-14 07:01:48,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,2019-11-11 05:28:05,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,2019-11-12 16:11:21,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [6]:
df_user['History'] = df_user['History'].str.split(" ")
df_user['History']

0         [N55189, N42782, N34694, N45794, N18445, N6330...
1         [N31739, N6072, N63045, N23979, N35656, N43353...
2         [N10732, N25792, N7563, N21087, N41087, N5445,...
3         [N45729, N2203, N871, N53880, N41375, N43142, ...
4                          [N10078, N56514, N14904, N33740]
                                ...                        
156960    [N7432, N58559, N1954, N43353, N14343, N13008,...
156961    [N9803, N104, N24462, N57318, N55743, N40526, ...
156962    [N29898, N59704, N4408, N9803, N53644, N26103,...
156963    [N4118, N47297, N3164, N43295, N6056, N38747, ...
156964                                     [N22997, N48742]
Name: History, Length: 156965, dtype: object

In [7]:
df_user = df_user[df_user['History'].str.len() >= 5].reset_index(drop=True)
df_user.head()

,Impression ID,User ID,Time,History,Impressions
0,1,U13740,2019-11-11 09:05:58,"[N55189, N42782, N34694, N45794, N18445, N6330...",N55689-1 N35729-0
1,2,U91836,2019-11-12 18:11:30,"[N31739, N6072, N63045, N23979, N35656, N43353...",N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,2019-11-14 07:01:48,"[N10732, N25792, N7563, N21087, N41087, N5445,...",N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,2019-11-11 05:28:05,"[N45729, N2203, N871, N53880, N41375, N43142, ...",N35729-0 N33632-0 N49685-1 N27581-0
4,6,U19739,2019-11-11 18:52:13,"[N39074, N14343, N32607, N32320, N22007, N442,...",N21119-1 N53696-0 N33619-1 N25722-0 N2869-0


In [8]:
## Padding and truncation of history itmes


## May change this max_lenght for model tuning process

max_length = 50 



def pad_or_truncate(history, max_length):
    if len(history) > max_length:
        return history[:max_length]
    else:
        return history + ['0'] * (max_length - len(history))
    
df_user['History'] = df_user['History'].apply(lambda x: pad_or_truncate(x, max_length))
df_user.head()

,Impression ID,User ID,Time,History,Impressions
0,1,U13740,2019-11-11 09:05:58,"[N55189, N42782, N34694, N45794, N18445, N6330...",N55689-1 N35729-0
1,2,U91836,2019-11-12 18:11:30,"[N31739, N6072, N63045, N23979, N35656, N43353...",N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,2019-11-14 07:01:48,"[N10732, N25792, N7563, N21087, N41087, N5445,...",N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,2019-11-11 05:28:05,"[N45729, N2203, N871, N53880, N41375, N43142, ...",N35729-0 N33632-0 N49685-1 N27581-0
4,6,U19739,2019-11-11 18:52:13,"[N39074, N14343, N32607, N32320, N22007, N442,...",N21119-1 N53696-0 N33619-1 N25722-0 N2869-0


In [9]:
df_user['History count'] = df_user['History'].apply(lambda x: len(x))
df_user['History count'].value_counts()

History count
50    136120
Name: count, dtype: int64

In [10]:
import numpy as np

df_news = df_news[['News ID', 'Category', 'SubCategory', 'Title', 'is_ai_news']]

news_dict = df_news.set_index('News ID').to_dict('index')


# Define a default dictionary for news not found.
default_dict = {'Category': 'unk', 'SubCategory': 'unk', 'Title': '', 'is_ai_news': np.nan}

# Step 2: Flatten the History column to a single list of all news IDs.
all_news_ids = [nid for sublist in df_user['History'] for nid in sublist]

# Step 3: Perform a vectorized lookup.
# This is a very fast operation on a single list.
all_features = [news_dict.get(nid, default_dict) for nid in all_news_ids]

# Step 4: Reconstruct the features into lists based on the original structure.
# Get the lengths of each original history list.
history_lengths = [len(hist) for hist in df_user['History']]

# Use a generator to chunk the features back into their respective lists.
def chunk_features(features, lengths):
    it = iter(features)
    for l in lengths:
        yield [next(it) for _ in range(l)]

df_user['History_features'] = list(chunk_features(all_features, history_lengths))


In [11]:
df_user['History_features'].iloc[0]

[{'Category': 'tv',
  'SubCategory': 'tvnews',
  'Title': "'Wheel Of Fortune' Guest Delivers Hilarious, Off The Rails Introduction",
  'is_ai_news': 0},
 {'Category': 'sports',
  'SubCategory': 'baseball_mlb',
  'Title': "Three takeaways from Yankees' ALCS Game 5 victory over the Astros",
  'is_ai_news': 0},
 {'Category': 'tv',
  'SubCategory': 'tvnews',
  'Title': "Rosie O'Donnell: Barbara Walters Isn't 'Up to Speaking to People' Right Now",
  'is_ai_news': 0},
 {'Category': 'news',
  'SubCategory': 'newscrime',
  'Title': "Four flight attendants were arrested in Miami's airport after bringing in thousands in cash, police say",
  'is_ai_news': 0},
 {'Category': 'sports',
  'SubCategory': 'football_ncaa',
  'Title': 'Michigan sends breakup tweet to Notre Dame as series goes on hold',
  'is_ai_news': 0},
 {'Category': 'lifestyle',
  'SubCategory': 'lifestylebuzz',
  'Title': "This Wedding Photo of a Canine Best Man Captures Just How Deep a Dog's Love Truly Is",
  'is_ai_news': 0},
 {'Ca

In [12]:
## Embedding the new id into integers for efficient processing

news_id_map = {id: idx for idx, id in enumerate(df_news['News ID'].unique())}

## mapping the news ids in history to integers
df_user['History'] = df_user['History'].apply(lambda x: [news_id_map.get(id, 0) for id in x])

In [13]:
## Impression preprocessing

def parse_impressions(impression):
    pairs = impression.split(' ')
    return [(pair.split('-')[0], int(pair.split('-')[1])) for pair in pairs]

df_user['Impressions'] = df_user['Impressions'].apply(lambda x: parse_impressions(x))

In [14]:
## exploding impressions
impression_df = df_user.explode('Impressions')

impression_df

,Impression ID,User ID,Time,History,Impressions,History count,History_features
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...","(N55689, 1)",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '..."
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...","(N35729, 0)",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '..."
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...","(N20678, 0)",50,"[{'Category': 'news', 'SubCategory': 'newscrim..."
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...","(N39317, 0)",50,"[{'Category': 'news', 'SubCategory': 'newscrim..."
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...","(N58114, 0)",50,"[{'Category': 'news', 'SubCategory': 'newscrim..."
...,...,...,...,...,...,...,...
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...","(N31556, 0)",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif..."
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...","(N23508, 0)",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif..."
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...","(N62805, 0)",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif..."
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...","(N27936, 0)",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif..."


In [15]:
impression_df['news_id'] = impression_df['Impressions'].apply(lambda x: x[0])
impression_df['label'] = impression_df['Impressions'].apply(lambda x: x[1])
impression_df = impression_df.drop(columns=['Impressions'])
impression_df

,Impression ID,User ID,Time,History,History count,History_features,news_id,label
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",N55689,1
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",N35729,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",N20678,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",N39317,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",N58114,0
...,...,...,...,...,...,...,...,...
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",N31556,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",N23508,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",N62805,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",N27936,0


In [16]:
impression_df['news_id'] = impression_df['news_id'].map(news_id_map).fillna(0).astype(int)
impression_df = impression_df.groupby('Impression ID').head(100)

In [17]:
impression_df

,Impression ID,User ID,Time,History,History count,History_features,news_id,label
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",33899,1
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",50688,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",37404,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",41305,0
1,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",34906,0
...,...,...,...,...,...,...,...,...
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33299,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",47049,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",39422,0
136119,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33649,0


In [18]:
training_df = impression_df.merge(
    df_user[['User ID', 'History', 'History_features', 'History count']],
    on='User ID',
    how='left'
).drop_duplicates(subset=['Impression ID', 'User ID', 'news_id', 'label']).reset_index(drop=True)

In [19]:
training_df = training_df.drop(columns=['History_y',
       'History_features_y', 'History count_y'])

In [20]:
training_df

,Impression ID,User ID,Time,History_x,History count_x,History_features_x,news_id,label
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",33899,1
1,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",50688,0
2,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",37404,0
3,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",41305,0
4,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",34906,0
...,...,...,...,...,...,...,...,...
4770798,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33299,0
4770799,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",47049,0
4770800,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",39422,0
4770801,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33649,0


---

In [21]:
df= training_df

df['user_id'] = df['User ID']
df['item_id'] = df['news_id']

In [22]:
df

,Impression ID,User ID,Time,History_x,History count_x,History_features_x,news_id,label,user_id,item_id
0,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",33899,1,U13740,33899
1,1,U13740,2019-11-11 09:05:58,"[6892, 10049, 15555, 21466, 26357, 4945, 14070...",50,"[{'Category': 'tv', 'SubCategory': 'tvnews', '...",50688,0,U13740,50688
2,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",37404,0,U91836,37404
3,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",41305,0,U91836,41305
4,2,U91836,2019-11-12 18:11:30,"[25815, 2333, 8523, 12086, 13462, 14201, 12732...",50,"[{'Category': 'news', 'SubCategory': 'newscrim...",34906,0,U91836,34906
...,...,...,...,...,...,...,...,...,...,...
4770798,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33299,0,U44625,33299
4770799,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",47049,0,U44625,47049
4770800,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",39422,0,U44625,39422
4770801,156964,U44625,2019-11-13 14:57:02,"[11410, 15584, 2260, 3681, 31665, 1131, 16301,...",50,"[{'Category': 'lifestyle', 'SubCategory': 'lif...",33649,0,U44625,33649


In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
from typing import Dict, Text

# Constants
EMBEDDING_DIM = 32
MAX_TOKENS = 5000
MAX_HISTORY_LEN = 10
BATCH_SIZE = 32

# Truncate history
df['History_x'] = df['History_x'].apply(lambda h: h[:MAX_HISTORY_LEN])
df['History_features_x'] = df['History_features_x'].apply(lambda h: h[:MAX_HISTORY_LEN])

# Create dataset
def create_dataset(df):
    item_features = df_news[['News ID', 'Category', 'SubCategory', 'Title', 'is_ai_news']].set_index('News ID').to_dict('index')
    
    dataset_dict = {
        'user_id': df['user_id'].astype(str).values,
        'item_id': df['item_id'].astype(str).values,
        'label': df['label'].astype(np.float32).values,
        'history': [np.array(h, dtype=np.int32) for h in df['History_x'].values],
        'history_features': [
            {
                'Category': [f['Category'] for f in hist],
                'SubCategory': [f['SubCategory'] for f in hist],
                'Title': [f['Title'] for f in hist],
            } for hist in df['History_features_x'].values
        ],
        'item_category': [item_features.get(str(id), {'Category': 'unk'})['Category'] for id in df['item_id'].values],
        'item_subcategory': [item_features.get(str(id), {'SubCategory': 'unk'})['SubCategory'] for id in df['item_id'].values],
        'item_title': [item_features.get(str(id), {'Title': ''})['Title'] for id in df['item_id'].values],
    }
    
    return tf.data.Dataset.from_tensor_slices(dataset_dict)

# Split data
train_size = int(0.8 * len(df))
train_df = df.sample(frac=1, random_state=42).iloc[:train_size]
test_df = df.iloc[train_size:]

train_dataset = create_dataset(train_df).batch(BATCH_SIZE)
test_dataset = create_dataset(test_df).batch(BATCH_SIZE)

# Vocabularies
unique_user_ids = df['user_id'].astype(str).unique()
unique_item_ids = df['item_id'].astype(str).unique()
unique_categories = df_news['Category'].astype(str).unique()
unique_subcategories = df_news['SubCategory'].astype(str).unique()

# Model
class NewsRecommendationModel(tfrs.Model):
    def __init__(self):
        super().__init__()

        # Embedding layers
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, EMBEDDING_DIM)
        ])
        
        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_item_ids) + 1, EMBEDDING_DIM)
        ])
        
        self.category_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_categories, mask_token=None),
            tf.keras.layers.Embedding(len(unique_categories) + 1, EMBEDDING_DIM)
        ])
        
        self.subcategory_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_subcategories, mask_token=None),
            tf.keras.layers.Embedding(len(unique_subcategories) + 1, EMBEDDING_DIM)
        ])
        
        self.title_vectorizer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS)
        self.title_vectorizer.adapt(df_news['Title'].values)
        self.title_embedding = tf.keras.Sequential([
            self.title_vectorizer,
            tf.keras.layers.Embedding(MAX_TOKENS, EMBEDDING_DIM),
            tf.keras.layers.GlobalAveragePooling1D()
        ])
        
        # Task
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_emb = self.user_embedding(features['user_id'])
        item_emb = self.item_embedding(features['item_id'])
        item_cat_emb = self.category_embedding(features['item_category'])
        item_subcat_emb = self.subcategory_embedding(features['item_subcategory'])
        item_title_emb = self.title_embedding(features['item_title'])

        item_combined = tf.concat([item_emb, item_cat_emb, item_subcat_emb, item_title_emb], axis=1)
        user_combined = user_emb  # History embeddings removed for simplicity

        x = tf.concat([user_combined, item_combined], axis=1)
        predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        return self.task(labels=features['label'], predictions=predictions)

# Train
model = NewsRecommendationModel()
model.compile(optimizer=tf.keras.optimizers.Adam(0.001))
model.fit(train_dataset, epochs=5, validation_data=test_dataset, verbose=1)

# Evaluate
metrics = model.evaluate(test_dataset, return_dict=True)
print("Evaluation metrics:", metrics)


2025-08-28 18:52:25.655412: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-08-28 18:52:25.659055: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-08-28 18:52:25.659514: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-08-28 18:52:25.659560: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-28 18:52:25.659825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.save('news_recommendation_model')